In [1]:
%config InlineBackend.figure_format = 'retina'

In [2]:
import random
import time

In [3]:
import tensorflow as tf
import numpy as np

import zmq

In [4]:
import learner
reload(learner)

<module 'learner' from 'learner.pyc'>

In [5]:
exp_socket_addr = "tcp://127.0.0.1:29884"
max_actions = 7
strat_socket_addr = "tcp://127.0.0.1:29885"

In [6]:
context = zmq.Context()
sock_exp = context.socket(zmq.REP)
sock_exp.bind(exp_socket_addr)

In [7]:
sock_strat = context.socket(zmq.PUB)
sock_strat.bind(strat_socket_addr)

In [8]:
NUM_HIDDEN = 128

In [37]:
def CreateNetworks(state, num_actions, scope, is_training=True):
    with tf.variable_scope(scope):
        hidden1 = tf.contrib.layers.relu(
            state, NUM_HIDDEN,
            weights_initializer=tf.truncated_normal_initializer(stddev=0.01),
            biases_initializer=tf.constant_initializer(0.01),
#                 normalizer_fn=tf.contrib.layers.batch_norm,
#                 normalizer_params={'is_training': is_training},
            scope='hidden1')
        hidden2 = tf.contrib.layers.relu(
            hidden1, NUM_HIDDEN,
            weights_initializer=tf.truncated_normal_initializer(stddev=0.01),
            biases_initializer=tf.constant_initializer(0.01),
#                 normalizer_fn=tf.contrib.layers.batch_norm,
#                 normalizer_params={'is_training': is_training},
            scope='hidden2')

        value = tf.contrib.layers.linear(hidden2, 1,
                                         weights_initializer=tf.truncated_normal_initializer(stddev=0.01),
                                         biases_initializer=tf.constant_initializer(0.),
                                         scope='value')
        value = tf.squeeze(value)
        if scope == 'model':
            logits = tf.contrib.layers.linear(hidden2, num_actions,
                                              weights_initializer=tf.truncated_normal_initializer(stddev=0.01),
                                              scope='policy')
        else:
            logits = None
    return logits, value

In [38]:
DEFAULT_OPTIONS = {
    'clip_grad': 3.,
    'learning_rate': 0.001,
    'update_steps': 10000,
}

In [39]:
class SupervisedPolicyValue(object):
    def __init__(self, build_networks, buf, options=DEFAULT_OPTIONS):
        self.options = options
        self.exp_buffer = buf
        with tf.device('/cpu:0'):
            self.state = tf.placeholder(tf.float32, shape=[None, self.exp_buffer.state_size], name='state')
            self.action = tf.placeholder(tf.int32, shape=[None], name='action')
            self.reward = tf.placeholder(tf.float32, shape=[None], name='reward')
            self.state1 = tf.placeholder(tf.float32, shape=[None, self.exp_buffer.state_size], name='state1')
            self.gamma = tf.placeholder(tf.float32, shape=[None], name='gamma')
            self.is_weights = tf.placeholder(tf.float32, shape=[None], name='is_weights')       
            self.is_training = tf.placeholder(tf.bool, shape=None, name='is_training')

            self.logits, self.baseline = build_networks(self.state,
                                                        is_training=self.is_training, scope='model')
            _,  self.baseline1 = build_networks(self.state1, is_training=False, scope='target')

            self.vars_pred = tf.get_collection(tf.GraphKeys.VARIABLES, 'model')
            self.vars_target = tf.get_collection(tf.GraphKeys.VARIABLES, 'target')

            self.copy_op = tf.group(
                *[tf.assign(y, x) for x, y in zip(self.vars_pred, self.vars_target)]
            )
            
#             self.tf_policy = tf.reshape(tf.multinomial(self.logits, 1), [])

            # Experimental
            self.rolled_baseline = tf.stop_gradient(self.reward + self.gamma * self.baseline1)
            self.advantage = self.rolled_baseline - self.baseline
            
            self.cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(self.logits, self.action)
            self.policy_loss = tf.reduce_mean(self.cross_entropy)
            # For actor-critic this should look like:
            # self.policy_loss = tf.reduce_mean(
            #     tf.mul(self.cross_entropy, tf.stop_gradient(self.advantage)))
            
            self.td_err_weight = tf.abs(self.advantage)
            self.value_loss = 0.5 * tf.reduce_mean(learner.HuberLoss(self.advantage, 5) * self.is_weights)
            self.policy_entropy = tf.reduce_mean(
                -tf.nn.softmax(self.logits) * tf.nn.log_softmax(self.logits))
            
            loss = self.policy_loss + 0.5 * self.value_loss - 0.01 * self.policy_entropy

            self.optimizer = tf.train.AdamOptimizer(options['learning_rate'])
            grads = self.optimizer.compute_gradients(loss, tf.get_collection(tf.GraphKeys.VARIABLES))
            if 'clip_grad' in options:
                grads = [(tf.clip_by_norm(g, options['clip_grad']) if g is not None else None, v)
                         for g, v in grads]

            for grad, var in grads:
                tf.histogram_summary('{}'.format(var.name), var)
                if grad is not None:
                    tf.histogram_summary('{}/grad'.format(var.name), grad)

            self.global_step = tf.Variable(0, name='global_step', trainable=False)
            self.train_op = self.optimizer.apply_gradients(grads, self.global_step)

            tf.histogram_summary('Monitor/TD Error', self.advantage)
            tf.histogram_summary('Monitor/Q', self.baseline)
            tf.histogram_summary("Monitor/Cross entropy loss", self.cross_entropy)
            tf.histogram_summary('Monitor/Weights', self.is_weights)
            tf.scalar_summary("Scalars/Loss", loss)
            tf.scalar_summary("Scalars/Q Func", tf.reduce_mean(self.baseline))
            tf.scalar_summary('Scalars/Weights', tf.reduce_mean(self.is_weights))
            tf.scalar_summary("Scalars/Actor loss", self.policy_loss)
            tf.scalar_summary("Scalars/Critic loss", self.value_loss)
            tf.scalar_summary("Scalars/TD error", tf.reduce_mean(self.td_err_weight))

            self.summary_op = tf.merge_all_summaries()

    def Init(self, sess, run_id):
        sess.run(tf.initialize_all_variables())
        self.writer = tf.train.SummaryWriter(
#             '/Users/vertix/tf/tensorflow_logs/aicup/%s'  % run_id
            '/media/vertix/UHDD/tmp/tensorflow_logs/aicup/%s' % run_id
        )
        self.saver = tf.train.Saver(tf.get_collection(tf.GraphKeys.VARIABLES))
        self.last_start = time.time()
        self.cur_step = 0
        self.writer.add_graph(tf.get_default_graph())

    def Step(self, sess, batch_size=32):
        idx, ss, aa, rr, ss1, gg, ww = self.exp_buffer.sample(batch_size)
        if ss is None:
            return
        
        feed_dict = {self.state: ss, self.action: aa, self.reward: rr, self.state1:ss1,
                     self.gamma: gg, self.is_weights: ww, self.is_training: True}

        if self.cur_step and self.cur_step % 100 != 0:
            self.cur_step, weights, _ = sess.run(
                [self.global_step, self.td_err_weight, self.train_op], feed_dict)
        else:
            self.cur_step, weights, _, smr = sess.run(
                [self.global_step, self.td_err_weight, self.train_op, self.summary_op], feed_dict)
            self.writer.add_summary(smr, self.cur_step)

        for ii, td_w in zip(idx, weights):
            self.exp_buffer.tree_update(ii, td_w)

        if self.cur_step % self.options['update_steps'] == 0:
            sess.run(self.copy_op)
            self.saver.save(sess, 'ac', global_step=self.global_step)
            if self.last_start is not None:
                self.writer.add_summary(
                    tf.Summary(
                        value=[tf.Summary.Value(
                                tag='Steps per sec',
                                simple_value=self.options['update_steps'] / (time.time() - self.last_start))]),
                    self.cur_step)
            self.last_start = time.time()

    def Stat(self, data):
        self.writer.add_summary(
            tf.Summary(
                value=[tf.Summary.Value(tag=name, simple_value=value)
                       for name, value in data.items()]), self.cur_step)

In [311]:
reload(learner)

<module 'learner' from 'learner.py'>

In [13]:
buf = learner.WeightedExperienceBuffer(0.6, 0.4, 100, buffer_size=1<<18)

In [16]:
for _ in range(1200):
    msg = sock_exp.recv_pyobj()
    sock_exp.send('Ok')
    if msg['type'] == 'exp':
        msg = msg['data']
        buf.add(msg['s'], msg['a'], msg['r'], msg['s1'], msg['g'], 100)

In [41]:
buf.inserted

311660

In [40]:
for i in xrange(min(buf.buffer_size, buf.inserted)):
    buf.tree_update(i, 100)

-------------------

In [43]:
sess.close()
tf.reset_default_graph()
sess = tf.InteractiveSession()

In [44]:
ac = SupervisedPolicyValue(lambda x, **kwargs: CreateNetworks(x, 7, **kwargs), buf, {
    'clip_grad': 3.,
    'learning_rate': 0.001,
    'update_steps': 10000,
})

In [45]:
ac.Init(sess, 'vi-1')

In [180]:
# ac.saver.restore(sess, 'ac-6260000')

In [ ]:
# %%prun

for i in range(15 * 10 ** 6):
# for i in range(5000):
    ac.Step(sess)
    if i % 10 == 0:
        msg = sock_exp.recv_pyobj()
        sock_exp.send('Ok')

        if msg['type'] == 'exp':
            msg = msg['data']
            ac.exp_buffer.add(msg['s'], msg['a'], msg['r'], msg['s1'], msg['g'], 100)
        elif msg['type'] == 'stat':
            ac.Stat(msg['data'])
    
#     if i % 20000 == 19999:
#         DumpVariables('network')

In [37]:
ac.exp_buffer.inserted

46960

In [45]:
_, s, a, r, s1, g, w = ac.exp_buffer.sample(15)

In [50]:
ac.exp_buffer.weight_min[1:20]

array([  0.07126157,   0.07126157,  15.84893192,   0.07126157,
        15.84893192,  15.84893192,  15.84893192,   0.07126157,
         0.07149289,  15.84893192,  15.84893192,  15.84893192,
        15.84893192,  15.84893192,  15.84893192,   0.07126157,
         0.07126157,   0.07149289,  15.84893192])

In [274]:
s.shape

(15, 814)

In [139]:
sess.run(ac.td_error, {ac.state: s, ac.state1: s1,  ac.reward: r, ac.gamma: g,
                       ac.is_training: False})

array([ 0.04569197,  0.0302515 , -0.13130021,  0.00646222, -0.08189249,
        0.01194906,  0.02656126, -0.10169649,  0.02854085, -0.10737276,
        0.02418947, -2.79416704, -0.14544439, -0.07501316,  0.0362587 ], dtype=float32)

In [140]:
sess.run(ac.baseline, {ac.state: s, ac.state1: s1,  ac.reward: r, ac.gamma: g,
                       ac.is_training: False})

array([ 5.08676052,  4.56864262,  2.74127364,  1.40395606,  4.47205591,
        3.96832752,  4.63735485,  4.69036865,  3.74232721,  4.2821207 ,
        4.1466217 ,  5.06364107,  2.63035035,  4.02202797,  4.52581549], dtype=float32)

In [141]:
sess.run(ac.baseline1, {ac.state: s, ac.state1: s1,  ac.reward: r, ac.gamma: g,
                       ac.is_training: False})

array([ 5.06640053,  4.56119728,  2.88700891,  1.40451646,  4.57683277,
        3.97625971,  4.63396358,  4.8161459 ,  3.73244858,  4.411551  ,
        4.14314795,  4.88221931,  2.78974342,  4.11762905,  4.51211739], dtype=float32)

In [148]:
for v in tf.get_collection(tf.GraphKeys.VARIABLES):
    print v.name

common/hidden1/weights:0
common/hidden1/biases:0
common/hidden2/weights:0
common/hidden2/biases:0
value/value/weights:0
value/value/biases:0
policy/policy/weights:0
policy/policy/biases:0
global_step:0
beta1_power:0
beta2_power:0
common/hidden1/weights/Adam:0
common/hidden1/weights/Adam_1:0
common/hidden1/biases/Adam:0
common/hidden1/biases/Adam_1:0
common/hidden2/weights/Adam:0
common/hidden2/weights/Adam_1:0
common/hidden2/biases/Adam:0
common/hidden2/biases/Adam_1:0
value/value/weights/Adam:0
value/value/weights/Adam_1:0
value/value/biases/Adam:0
value/value/biases/Adam_1:0
policy/policy/weights/Adam:0
policy/policy/weights/Adam_1:0
policy/policy/biases/Adam:0
policy/policy/biases/Adam_1:0


In [224]:
import cPickle

def DumpVariables(filename):
    network_vars = {}
    for v in tf.get_collection(tf.GraphKeys.VARIABLES):
        if '/' in v.name and 'Adam' not in v.name:
            network_vars[v.name] = v.eval()
    with open(filename, 'w') as f:
        cPickle.dump(network_vars, f)        

In [225]:
DumpVariables('network')

----------------

In [162]:
def ReLu(x):
    return np.maximum(x, 0)

In [163]:
def BatchNorm(state, network_vars, key):
    eps = 0.001
    inv = 1.0 / np.sqrt(network_vars[key + '/moving_variance:0'] + eps)

    return state * inv + (network_vars[key + '/beta:0'] - network_vars[key + '/moving_mean:0'] * inv)

In [164]:
def Softmax(state):
    state -= np.max(state)
    e = np.exp(state)
    return e / np.sum(e)

In [205]:
class NNPolicy(object):
    def __init__(self, network_vars):
        self.vars = network_vars
        self.actions = None

    def Logits(self, state):
        state = np.matmul(state, self.vars['common/hidden1/weights:0'])
        state += self.vars['common/hidden1/biases:0']
        state = ReLu(state)

        state = np.matmul(state, self.vars['common/hidden2/weights:0'])
        state += self.vars['common/hidden2/biases:0']
        state = ReLu(state)

        logits = np.matmul(state, self.vars['policy/policy/weights:0'])
        logits += self.vars['policy/policy/biases:0']
        return logits
    
    def Softmax(self, state):
        logits = self.Logits(state)
        return Softmax(logits)
    
    def Sample(self, state):
        sm = self.Softmax(state)
        if self.actions is None:
            self.actions = range(len(sm))
        return np.random.choice(self.actions, p=sm)

In [206]:
nnp = NNPolicy(network_vars)

In [219]:
nnp.Softmax(s[1, :])

array([  5.60286788e-05,   1.27642462e-75,   1.33276611e-08,
         5.64379590e-10,   8.89499400e-01,   1.10444557e-01,
         1.42909448e-18])

In [218]:
nnp.Sample(s[1, :])

4

In [178]:
np.max(nnp.Logits(s) -  ac.logits.eval({ac.state: s}))

0.00021755436517878479

In [173]:
ac.logits.eval({ac.state: s})

(15, 7)

In [280]:
ql.pred_vars[6].eval({ql.state: ss, ql.is_training: False})

array([[ 0.4263148 ,  0.27988333,  0.52581638,  0.42325473,  0.46426272,
         0.47350475,  0.352005  ]], dtype=float32)

In [284]:
QFunction(network_vars).Q(ss)

array([[ 0.42631482,  0.27988335,  0.52581639,  0.4232547 ,  0.46426274,
         0.47350477,  0.352005  ]])

In [170]:
ql.q.eval(feed_dict)

array([ 1746.92163086,     0.        ,     0.        ,     0.        ,
           0.        ,     0.        ,     0.        ,     0.        ,
           0.        ,  3156.87939453], dtype=float32)

In [174]:
ql.q_s1.eval(feed_dict)

array([ 10.50465488,  10.50465488,  10.50465488,  10.50465488,
        10.50465488,  10.50465488,  10.50465488,  10.50465488,
        10.50465488,  10.50465488], dtype=float32)

In [173]:
sess.run(ql.copy_op)

In [138]:
Select(ql.pred_vars[2], ql.action).eval(feed_dict)

array([ 2186.74389648,  6496.70410156,  2797.15771484,  4662.953125  ,
           0.        ,  2761.18579102,  3284.98608398,  6556.95996094,
        2748.39379883,     0.        ], dtype=float32)

In [148]:
tf.reduce_sum(ql.pred_vars[2] * tf.one_hot(ql.action, ql.num_actions), reduction_indices=1).eval(feed_dict)

array([ 2186.74389648,  6496.70410156,  2797.15771484,  4662.953125  ,
           0.        ,  2761.18579102,  3284.98608398,  6556.95996094,
        2748.39379883,     0.        ], dtype=float32)

In [93]:
tf.shape(ql.pred_vars[2]).eval(feed_dict)

array([10,  7], dtype=int32)

In [51]:
ql.delta.eval()

array([ 25.,  25.,  25.,  25.,  25.,  25.,  25.,  25.,  25.,  25.], dtype=float32)

In [40]:
tf.reduce_mean(ql.delta).eval({ql.state: ss, ql.action: aa, ql.reward: rr, ql.state1:ss1, ql.gamma: gg})

25.0

In [283]:
ql.q.eval({ql.state: ss, ql.action: aa})

array([    0.        ,     0.        ,  2561.84985352,  2696.66064453,
        3319.86474609,  2376.26171875,  2375.80859375,  2602.42016602,
        2548.83056641,     0.        ], dtype=float32)

In [284]:
ql.q_s1.eval({ql.state1: ss1})

array([ 9.87346268,  9.87346268,  9.87346268,  9.87346268,  9.87346268,
        9.87346268,  9.87346268,  9.87346268,  9.87346268,  9.87346268], dtype=float32)